// WARNING: Run with Scala kernel from Variation8a (in jupyter-lab)

# Variation 8b. QUEΛ to SQL 

Warm up exercise: a string representation for QUEΛ queries

In [24]:
abstract class StringRep[T] extends (String => Int => String)

object StringRep{
    
     def apply[T](f: String => Int => String): StringRep[T] = 
        new StringRep[T]{
            def apply(i: String): Int => String = f(i)
        }

    implicit object StringQUEΛ extends QUEΛ[StringRep]{
        // base types

        def bool(b: Boolean): StringRep[Boolean] = 
            StringRep(_ => _ => b.toString)

        def int(i: Int): StringRep[Int] = 
            StringRep(_ => _ => i.toString)

        def str(s: String): StringRep[String] = 
            StringRep(_ => _ => s)

        def >(i1: StringRep[Int], i2: StringRep[Int]): StringRep[Boolean] = 
            StringRep(t => i => s"${i1(t)(i)} > ${i2(t)(i)}")

        // ADTs

        def tuple2[A, B](a: StringRep[A], b: StringRep[B]): StringRep[(A, B)] = 
            StringRep(t => i => s"""(${a(t)(i)}, ${b(t)(i)})""")

        def none[A]: StringRep[Option[A]] = 
            StringRep(_ => _ => "none")

        def some[A](a: StringRep[A]): StringRep[Option[A]] = 
            StringRep(t => i => s"some(${a(t)(i)})")

        def exists[A](o: StringRep[Option[A]])(cond: StringRep[A] => StringRep[Boolean]): StringRep[Boolean] = 
            StringRep(t => i => s"""exists(${o(t)(i)})(x$i => ${cond(_ => _ => s"x$i")(t)(i)})""")

        // Comprehensions

        def bind[A, B](q: StringRep[List[A]])(f: StringRep[A] => StringRep[List[B]]): StringRep[List[B]] = 
            StringRep(t => i => 
                s"""|${t}bind(
                    |${q(t+"    ")(i)}
                    |${t}){ x$i => 
                    |${f(_ => _ => s"x$i")(t+"    ")(i+1)}
                    |${t}}""".stripMargin)

        def pure[A](a: StringRep[A]): StringRep[List[A]] = 
            StringRep(t => i => t + s"pure(${a(t)(i)})")

        def where[A](cond: StringRep[Boolean])(q: StringRep[List[A]]): StringRep[List[A]] = 
            StringRep(t => i => 
                s"""|${t}where(${cond(t)(i)})(
                    |${q(t+"    ")(i)}
                    |${t})""".stripMargin)

        // Equality

        def ===[A](a1: StringRep[A], a2: StringRep[A]): StringRep[Boolean] = 
            StringRep(t => i => s"===(${a1(t)(i)}, ${a2(t)(i)})")
    }
    
    object CityStringRep extends CityModel[StringRep]{
        def id(city: StringRep[City]): StringRep[Int] = 
            StringRep(t => i => s"id(${city(t)(i)})")

        def name(city: StringRep[City]): StringRep[String] = 
            StringRep(t => i => s"cityName(${city(t)(i)})")

        def country(city: StringRep[City]): StringRep[String] = 
            StringRep(t => i => s"country(${city(t)(i)})")

        def population(city: StringRep[City]): StringRep[Int] = 
            StringRep(t => i => s"population(${city(t)(i)})")
    }

    object CountryStringRep extends CountryModel[StringRep]{
        def code(country: StringRep[Country]): StringRep[String] = 
            StringRep(t => i => s"code(${country(t)(i)})")

        def name(country: StringRep[Country]): StringRep[String] = 
            StringRep(t => i => s"name(${country(t)(i)})")

        def capital(country: StringRep[Country]): StringRep[Option[Int]] = 
            StringRep(t => i => s"capital(${country(t)(i)})")
    }

    implicit object WorldStringRep extends WorldModel[StringRep]{
        val Ci = CityStringRep
        val Co = CountryStringRep

        def countries: StringRep[List[Country]] = 
            StringRep(t => _ => t + s"countries")

        def cities: StringRep[List[City]] = 
            StringRep(t => _ => t + s"cities")
    }
}

defined class StringRep
defined object StringRep

In [25]:
println(largeCapitals[StringRep].apply("")(0))

bind(
    countries
){ x0 => 
    bind(
        bind(
            bind(
                cities
            ){ x1 => 
                where(exists(capital(x0))(x2 => ===(x2, id(x1))))(
                    pure(x1)
                )
            }
        ){ x1 => 
            where(population(x1) > 8000000)(
                pure(x1)
            )
        }
    ){ x1 => 
        pure((cityName(x1), name(x0)))
    }
}


In [26]:
def largeCapitalsNormalized[Repr[_]](implicit Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[(String, String)]] = 
    W.countries.flatMap{ x0 => 
        W.cities.flatMap{ x1 => 
            Q.where(capital(x0).exists(x2 => x2 === id(x1)))(
                Q.where(population(x1) > Q.int(8000000))(
                    Q.pure((cityName(x1), name(x0)))
                )
            )
        }
    }

defined function largeCapitalsNormalized

In [27]:
println(largeCapitalsNormalized[StringRep].apply("")(0))

bind(
    countries
){ x0 => 
    bind(
        cities
    ){ x1 => 
        where(exists(capital(x0))(x2 => ===(x2, id(x1))))(
            where(population(x1) > 8000000)(
                pure((cityName(x1), name(x0)))
            )
        )
    }
}


## SQL interpreter

In [28]:
sealed abstract class DoobieRep[T]
case class SQLVar[A](fragment: Fragment) extends DoobieRep[A]
case class SQLInt(fragment: Fragment) extends DoobieRep[Int]
case class SQLBoolean(fragment: Fragment) extends DoobieRep[Boolean]
case class SQLString(fragment: Fragment) extends DoobieRep[String]
case class SQLOption[A](rep: DoobieRep[A]) extends DoobieRep[Option[A]] // TBD: Option[DoobieRep[A]]
case class SQLTuple[A, B](_1: DoobieRep[A], _2: DoobieRep[B]) extends DoobieRep[(A, B)]
case class SQLStatement[A](
    from: List[DoobieRep.Table], 
    where: List[DoobieRep[Boolean]], 
    select: DoobieRep[A]) extends DoobieRep[List[A]]

object DoobieRep extends ModelConstructors with QueryInterpreter with QUEΛInterpreter

trait ModelConstructors{ self: DoobieRep.type => 

    case class Table(name: String, varName: String)
    
    object Table{
        def fragment(table: Table): Fragment = 
            Fragment.const(table.name) ++ Fragment.const0(table.varName)
    }
        
    def statement[A](table: Table): SQLStatement[A] = 
        SQLStatement(List(table), List(), SQLVar(Fragment.const0(table.varName)))
    
    def intSelector[A](d: DoobieRep[A], field: String): DoobieRep[Int] = 
        SQLInt(d.fragment ++ Fragment.const0(s".$field"))

    def stringSelector[A](d: DoobieRep[A], field: String): DoobieRep[String] = 
        SQLString(d.fragment ++ Fragment.const0(s".$field"))

    def boolSelector[A](d: DoobieRep[A], field: String): DoobieRep[Boolean] = 
        SQLBoolean(d.fragment ++ Fragment.const0(s".field"))

}

trait QueryInterpreter{
    
    import doobie.util._, query._
    import cats.evidence.Is

    trait IsList[L]{
        type A
        val read: Read[A]
    }

    object IsList{
        type Aux[L, _A] = IsList[L]{ type A = _A }

        implicit def isList[_A: Read]: IsList.Aux[List[_A], _A] = 
            new IsList[List[_A]]{
                type A = _A
                val read = Read[_A]
            }
    }
    
    implicit class FragmentDoobieRep[T](d: DoobieRep[T]){ 
        val fragment: Fragment = d match {
            case SQLVar(fragment) => 
                fragment
            
            case d: SQLInt => 
                d.fragment
            
            case d: SQLBoolean => 
                d.fragment
            
            case d: SQLString => 
                d.fragment
            
            case d: SQLOption[_] => 
                d.rep.fragment
            
            case d: SQLTuple[_, _] => 
                d._1.fragment ++ fr0", " ++ d._2.fragment
            
            case d: SQLStatement[_] => {
                val fromFr: List[Fragment] = 
                    if (d.from.isEmpty) List() 
                    else List(fr"from" ++ d.from.map(DoobieRep.Table.fragment).mkFragment(fr0",", false))
                val whereFr: List[Fragment] = 
                    if (d.where.isEmpty) List() 
                    else List(fr"where" ++ d.where.map(_.fragment).mkFragment(fr0"and"))
                (List(fr"select" ++ d.select.fragment) ++ fromFr ++ whereFr).mkFragment(fr0" ", false, false)
            }
        }
        
        def query[A](implicit ev: IsList.Aux[T, A]): Query0[A] = 
            fragment.query[A](ev.read)
    }
}

trait QUEΛInterpreter{
    
    implicit object QUEΛ extends QUEΛ[DoobieRep]{
        // base types

        def bool(b: Boolean): DoobieRep[Boolean] = 
            SQLBoolean(Fragment.const(b.toString))

        def int(i: Int): DoobieRep[Int] = 
            SQLInt(Fragment.const0(i.toString))

        def str(s: String): DoobieRep[String] = 
            SQLString(Fragment.const(s))

        def >(i1: DoobieRep[Int], i2: DoobieRep[Int]): DoobieRep[Boolean] = 
            SQLBoolean(i1.fragment ++ fr0" > " ++ i2.fragment)

        // ADTs

        def tuple2[A, B](a: DoobieRep[A], b: DoobieRep[B]): DoobieRep[(A, B)] = 
            SQLTuple(a, b) 

        def none[A]: DoobieRep[Option[A]] = 
            ???

        def some[A](a: DoobieRep[A]): DoobieRep[Option[A]] = 
            SQLOption(a)

        def exists[A](o: DoobieRep[Option[A]])(cond: DoobieRep[A] => DoobieRep[Boolean]): DoobieRep[Boolean] = {
            val SQLOption(a) = o
            cond(a)
        }

        // Comprehensions

        def bind[A, B](q: DoobieRep[List[A]])(f: DoobieRep[A] => DoobieRep[List[B]]): DoobieRep[List[B]] = {
            val SQLStatement(from1, where1, select1) = q
            val SQLStatement(from2, where2, select2) = f(select1)
            SQLStatement(from1 ++ from2, where1 ++ where2, select2)
        }

        def pure[A](a: DoobieRep[A]): DoobieRep[List[A]] = 
            SQLStatement(List(), List(), a)

        def where[A](cond: DoobieRep[Boolean])(q: DoobieRep[List[A]]): DoobieRep[List[A]] = {
            val SQLStatement(from, where, select) = q
            SQLStatement(from, cond :: where, select)
        }        
        
        // Equality

        def ===[A](a1: DoobieRep[A], a2: DoobieRep[A]): DoobieRep[Boolean] = 
            SQLBoolean(a1.fragment ++ fr" =" ++ a2.fragment)
    }
    
    object CityDoobieRep extends CityModel[DoobieRep]{

        def id(city: DoobieRep[City]): DoobieRep[Int] = 
            DoobieRep.intSelector(city, "id")

        def name(city: DoobieRep[City]): DoobieRep[String] = 
            DoobieRep.stringSelector(city, "name")

        def country(city: DoobieRep[City]): DoobieRep[String] = 
            DoobieRep.stringSelector(city, "country")

        def population(city: DoobieRep[City]): DoobieRep[Int] = 
            DoobieRep.intSelector(city, "population")
    }

    object CountryDoobieRep extends CountryModel[DoobieRep]{

        def code(country: DoobieRep[Country]): DoobieRep[String] = 
            DoobieRep.stringSelector(country, "code")

        def name(country: DoobieRep[Country]): DoobieRep[String] = 
            DoobieRep.stringSelector(country, "name")

        def capital(country: DoobieRep[Country]): DoobieRep[Option[Int]] = 
            SQLOption(DoobieRep.intSelector(country, "capital"))
    }

    implicit object WorldDoobieRep extends WorldModel[DoobieRep]{
        val Ci = CityDoobieRep
        val Co = CountryDoobieRep

        def countries: DoobieRep[List[Country]] =
            DoobieRep.statement(DoobieRep.Table("country", "x0"))

        def cities: DoobieRep[List[City]] = 
            DoobieRep.statement(DoobieRep.Table("city", "x1"))
    }
}

defined class DoobieRep
defined class SQLVar
defined class SQLInt
defined class SQLBoolean
defined class SQLString
defined class SQLOption
defined class SQLTuple
defined class SQLStatement
defined object DoobieRep
defined trait ModelConstructors
defined trait QueryInterpreter
defined trait QUEΛInterpreter

In [29]:
largeCapitals[DoobieRep].query.to[List].transact(xa).unsafeRunSync.timed
largeCapitalsNormalized[DoobieRep].fragment
largeCapitals[DoobieRep].fragment

163 millis


res28_0: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)
res28_1: Fragment = Fragment("select x1.name, x0.name from country x0, city x1 where x0.capital = x1.id and x1.population > 8000000")
res28_2: Fragment = Fragment("select x1.name, x0.name from country x0, city x1 where x0.capital = x1.id and x1.population > 8000000")

### Modular queries

In [30]:
def city[Repr[_]: QUEΛ: WorldModel](idCity: Repr[Option[Int]]): Repr[List[City]] = 
    for {
        c <- cities if idCity.exists{ _ === id(c) }
    } yield c

def largeCity[Repr[_]: QUEΛ: WorldModel](idCity: Repr[Option[Int]]): Repr[List[City]] = 
    for {
        c <- city(idCity)
        if population(c) > 8000000
    } yield c

def largeCapitalsModular[Repr[_]: QUEΛ: WorldModel]: Repr[List[(Int, (String, String))]] = 
    for {
        country <- countries
        city <- largeCity(capital(country))
    } yield (id(city), (cityName(city), name(country)): Repr[(String, String)])

defined function city
defined function largeCity
defined function largeCapitalsModular

In [38]:
largeCapitalsModular[DoobieRep].fragment
largeCapitalsModular[DoobieRep].query.to[List].transact(xa).unsafeRunSync.timed

37 millis


res37_0: Fragment = Fragment("select x1.id, x1.name, x0.name from country x0, city x1 where x0.capital = x1.id and x1.population > 8000000")
res37_1: List[(Int, (String, String))] = List(
  (939, ("Jakarta", "Indonesia")),
  (2331, ("Seoul", "South Korea")),
  (2515, ("Ciudad de M\u00e9xico", "Mexico")),
  (3580, ("Moscow", "Russian Federation"))
)

In [32]:
xa.trans.apply(largeCapitalsModular[DoobieRep].query.to[List]).unsafeRunSync.timed

31 millis


res31: List[(Int, (String, String))] = List(
  (939, ("Jakarta", "Indonesia")),
  (2331, ("Seoul", "South Korea")),
  (2515, ("Ciudad de M\u00e9xico", "Mexico")),
  (3580, ("Moscow", "Russian Federation"))
)

In [33]:
xa.rawTrans.apply(largeCapitalsModular[DoobieRep].query.to[List]).unsafeRunSync.timed

34 millis


res32: List[(Int, (String, String))] = List(
  (939, ("Jakarta", "Indonesia")),
  (2331, ("Seoul", "South Korea")),
  (2515, ("Ciudad de M\u00e9xico", "Mexico")),
  (3580, ("Moscow", "Russian Federation"))
)

### All fields .*

TBD

In [35]:
largeCity[DoobieRep](DoobieRep.QUEΛ.some(DoobieRep.QUEΛ.int(939))) // query.to[List].transact(xa).unsafeRunSync

res34: DoobieRep[List[City]] = SQLStatement(
  List(Table("city", "x1")),
  List(
    SQLBoolean(Fragment("939 = x1.id")),
    SQLBoolean(Fragment("x1.population > 8000000"))
  ),
  SQLVar(Fragment("x1"))
)

In [36]:
sql"select x1.*, x1.* from city x1 where 939 = x1.id".query[(City, City)].to[List].transact(xa).unsafeRunSync

res35: List[(City, City)] = List(
  (
    City(939, "Jakarta", "IDN", "Jakarta Raya", 9604900),
    City(939, "Jakarta", "IDN", "Jakarta Raya", 9604900)
  )
)

In [37]:
sql"select row(x1), x1, x1.* from city x1 where 939 = x1.id".query[(String, String, City)].to[List].transact(xa).unsafeRunSync

res36: List[(String, String, City)] = List(
  (
    "(\"(939,Jakarta,IDN,\"\"Jakarta Raya\"\",9604900)\")",
    "(939,Jakarta,IDN,\"Jakarta Raya\",9604900)",
    City(939, "Jakarta", "IDN", "Jakarta Raya", 9604900)
  )
)